In [127]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce

In [128]:
dtypes = {'id': 'int32', 
'titulo': 'object', 
'descripcion': 'object', 
'tipodepropiedad': 'category', 
'direccion': 'object', 
'ciudad': 'object', 
'provincia': 'category', 
'antiguedad': 'float', 
'habitaciones': 'float', 
'garages': 'float', 
'banos': 'float', 
'metroscubiertos': 'float', 
'metrostotales': 'float', 
'idzona': 'object', 
'lat': 'float64', 
'lng': 'float64', 
'gimnasio': 'float', 
'usosmultiples': 'float', 
'piscina': 'float', 
'escuelascercanas': 'float', 
'centroscomercialescercanos': 'float', 
'precio': 'float', }

data = pd.read_csv("train.csv", dtype = dtypes)
data.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


# Pre-procesamiento de data

In [129]:
data["ciudad"].nunique()

875

In [130]:
data.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [131]:
data.isnull().sum().sum()

535127

In [132]:
data.size

5520000

In [133]:
(data.isnull().sum().sum()/(data.size))*100

9.694329710144928

Los Nulls representan el 10% de los datos.

# CON XGBOOST NO HACE FALTA PREOCUPARSE POR LOS NULLs

# Levanto el csv de test para calcularle los features en paralelo

In [134]:
test = pd.read_csv("test.csv", dtype = dtypes)
test.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,...,NaN,NaN,19.408668,-99.246767,2013-07-20 00:00:00,0.0,0.0,0.0,0.0,0.0
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,67.0,113851.0,21.032480,-89.592424,2015-10-24 00:00:00,0.0,0.0,0.0,0.0,0.0
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,100.0,23620.0,19.332829,-99.152913,2015-05-30 00:00:00,0.0,0.0,0.0,0.0,1.0
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,86.0,129347.0,16.860487,-99.878383,2015-04-02 00:00:00,0.0,0.0,0.0,0.0,0.0
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,76.0,57125.0,19.640482,-99.127273,2013-08-15 00:00:00,0.0,0.0,0.0,1.0,1.0


In [135]:
len(test)

60000

# Preparacion del set de datos de entrenamiento (features)

La idea es preparar el set con los datos para exportar y que despues el modelo simplemente levante, separe en set de entrenamiento y test, entrene y devuelva una prediccion.

Se procede a calcular features. Cada feature se agregara al DataFrame final que tendra que levantar despues el modelo. Ojo que dentro de este DataFrame tambien va a estar el precio (que es el label).

In [136]:
train_set = pd.DataFrame()

In [137]:
test_set = pd.DataFrame()

Primero de todo, se agrega el id y el precio asocidado. A partir de esto, se agregaran los features que se consideren necesarios.

In [138]:
train_set["id"] = data["id"]
train_set["precio"] = data["precio"]

In [139]:
train_set.head()

,id,precio
0,254099,2273000.0
1,53461,3600000.0
2,247984,1200000.0
3,209067,650000.0
4,185997,1150000.0


A partir de esto se generan features.

Igual pero para test_set. Solo que sin el precio.

In [140]:
test_set["id"] = test["id"]

In [141]:
test_set.head()

,id
0,4941
1,51775
2,115253
3,299321
4,173570


In [142]:
len(test_set)

60000

# Se codifican las variables categoricas

In [143]:
data['train'] = True
test['train'] = False
combined = pd.concat([data, test], sort = True)

In [144]:
train_set['train'] = True
test_set["train"] = False
combined_set = pd.concat([train_set, test_set], sort = False)

In [145]:
len(combined_set)

300000

In [146]:
combined['tipodepropiedad'].nunique()

24

In [147]:
combined['ciudad'].nunique()

921

Ciudad tiene demasiados valores posibles para OneHotEncoding

In [148]:
combined['provincia'].nunique()

32

In [149]:
combined_set['tipodepropiedad'] = combined['tipodepropiedad']
combined_set['provincia'] = combined['provincia']

In [150]:
len(combined_set)

300000

In [151]:
var_categoricas = ['tipodepropiedad', 'provincia']

In [152]:
one_hot_enc = ce.OneHotEncoder(handle_unknown = 'ignore')
one_hot_encoded = one_hot_enc.fit_transform(combined_set[var_categoricas])
one_hot_encoded.columns

Index(['tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'provincia_20', 'provincia_21', 'provincia_22', 'provincia_23',
  

In [153]:
combined_set = combined_set.join(one_hot_encoded)
combined_set.columns

Index(['id', 'precio', 'train', 'tipodepropiedad', 'provincia',
       'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'p

In [154]:
len(combined_set)

420000

In [155]:
len(train_set)

240000

In [156]:
len(test_set)

60000

In [157]:
train_set = combined_set[combined_set["train"] == True]
test_set = combined_set[combined_set["train"] == False]

In [158]:
len(train_set)

300000

In [159]:
len(test_set)

120000

In [160]:
train_set = train_set.drop(columns = ['tipodepropiedad', 'provincia' , 'train'])

In [161]:
train_set.columns

Index(['id', 'precio', 'tipodepropiedad_1', 'tipodepropiedad_2',
       'tipodepropiedad_3', 'tipodepropiedad_4', 'tipodepropiedad_5',
       'tipodepropiedad_6', 'tipodepropiedad_7', 'tipodepropiedad_8',
       'tipodepropiedad_9', 'tipodepropiedad_10', 'tipodepropiedad_11',
       'tipodepropiedad_12', 'tipodepropiedad_13', 'tipodepropiedad_14',
       'tipodepropiedad_15', 'tipodepropiedad_16', 'tipodepropiedad_17',
       'tipodepropiedad_18', 'tipodepropiedad_19', 'tipodepropiedad_20',
       'tipodepropiedad_21', 'tipodepropiedad_22', 'tipodepropiedad_23',
       'tipodepropiedad_24', 'tipodepropiedad_25', 'provincia_1',
       'provincia_2', 'provincia_3', 'provincia_4', 'provincia_5',
       'provincia_6', 'provincia_7', 'provincia_8', 'provincia_9',
       'provincia_10', 'provincia_11', 'provincia_12', 'provincia_13',
       'provincia_14', 'provincia_15', 'provincia_16', 'provincia_17',
       'provincia_18', 'provincia_19', 'provincia_20', 'provincia_21',
       'provincia_

In [162]:
test_set = test_set.drop(columns = ['tipodepropiedad', 'precio' , 'provincia', 'train'])

In [163]:
test_set.columns

Index(['id', 'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'provincia_20', 'provincia_21', 'provincia_22', 'provincia_2

In [164]:
len(test)

60000

In [165]:
len(test_set)

120000

In [166]:
test_set = test_set.drop_duplicates("id")

In [167]:
len(test_set)

60000

In [168]:
train_set = train_set.drop_duplicates("id")

In [169]:
len(train_set)

240000

## Se agregan todas las columnas numericas que ya se tienen

In [170]:
data.dtypes

id                               int32
titulo                          object
descripcion                     object
tipodepropiedad               category
direccion                       object
ciudad                          object
provincia                     category
antiguedad                     float64
habitaciones                   float64
garages                        float64
banos                          float64
metroscubiertos                float64
metrostotales                  float64
idzona                          object
lat                            float64
lng                            float64
fecha                           object
gimnasio                       float64
usosmultiples                  float64
piscina                        float64
escuelascercanas               float64
centroscomercialescercanos     float64
precio                         float64
train                             bool
dtype: object

In [171]:
data_num = data.loc[:,["id", "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [172]:
train_set = train_set.merge(data_num, on = ["id"], how = "inner")
train_set.head()

,id,precio,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,...,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,254099,2273000.0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,53461,3600000.0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,247984,1200000.0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,209067,650000.0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,185997,1150000.0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [173]:
test_num = test.loc[:,["id", "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [174]:
test_set = test_set.merge(test_num, on = ["id"], how = "inner")
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,51775,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,115253,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,299321,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,173570,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


## Se agregan el resto de las variables

In [175]:
data_num_fill = data.loc[:, ["id", "antiguedad", "metrostotales", "metroscubiertos", 'garages', 'banos', 'lat', 'lng']]

In [176]:
train_set = train_set.merge(data_num_fill, on = 'id', how = 'inner')
train_set.head()

,id,precio,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,...,piscina,escuelascercanas,centroscomercialescercanos,antiguedad,metrostotales,metroscubiertos,garages,banos,lat,lng
0,254099,2273000.0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,NaN,80.0,80.0,1.0,2.0,NaN,NaN
1,53461,3600000.0,0,1,0,0,0,0,0,0,...,0.0,1.0,1.0,10.0,180.0,268.0,2.0,2.0,19.310205,-99.227655
2,247984,1200000.0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,5.0,166.0,144.0,2.0,2.0,NaN,NaN
3,209067,650000.0,0,0,1,0,0,0,0,0,...,0.0,1.0,1.0,1.0,67.0,63.0,1.0,1.0,19.301890,-99.688015
4,185997,1150000.0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,10.0,95.0,95.0,1.0,1.0,NaN,NaN


In [177]:
test_num_fill = test.loc[:, ["id", "antiguedad", "metrostotales", "metroscubiertos", 'garages', 'banos', 'lat', 'lng']]

In [178]:
test_set = test_set.merge(test_num_fill, on = 'id', how = 'inner')
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,piscina,escuelascercanas,centroscomercialescercanos,antiguedad,metrostotales,metroscubiertos,garages,banos,lat,lng
0,4941,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,29.0,NaN,300.0,NaN,4.0,19.408668,-99.246767
1,51775,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,NaN,67.0,67.0,1.0,1.0,21.032480,-89.592424
2,115253,0,0,1,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,100.0,87.0,1.0,2.0,19.332829,-99.152913
3,299321,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,2.0,86.0,86.0,2.0,2.0,16.860487,-99.878383
4,173570,1,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,10.0,76.0,80.0,1.0,1.0,19.640482,-99.127273


In [179]:
len(test_set)

60000

In [180]:
train_set.isnull().sum().sum()

428344

# Nuevos features

# Promedio de metroscubiertos por tipo de propiedad

In [181]:
metros_mean = data.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "mean"}).reset_index()

In [182]:
metros_mean.head()

,tipodepropiedad,metroscubiertos
0,Apartamento,117.048657
1,Bodega comercial,251.153639
2,Casa,194.026911
3,Casa en condominio,187.697231
4,Casa uso de suelo,254.068966


In [183]:
metros_mean.rename(columns = {"metroscubiertos" : "metroscubiertos_mean_tipodeprop"}, inplace = True)
metros_mean.head()

,tipodepropiedad,metroscubiertos_mean_tipodeprop
0,Apartamento,117.048657
1,Bodega comercial,251.153639
2,Casa,194.026911
3,Casa en condominio,187.697231
4,Casa uso de suelo,254.068966


In [184]:
data = data.merge(metros_mean, on = 'tipodepropiedad')

In [185]:
train_set = train_set.merge(data[['id', 'metroscubiertos_mean_tipodeprop']], on = "id")

### Lo mismo pero para test

In [186]:
metros_mean = test.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "mean"}).reset_index()

In [187]:
metros_mean.head()

,tipodepropiedad,metroscubiertos
0,Apartamento,116.138123
1,Bodega comercial,250.683849
2,Casa,193.814888
3,Casa en condominio,186.547166
4,Casa uso de suelo,241.857895


In [188]:
metros_mean.rename(columns = {"metroscubiertos" : "metroscubiertos_mean_tipodeprop"}, inplace = True)
metros_mean.head()

,tipodepropiedad,metroscubiertos_mean_tipodeprop
0,Apartamento,116.138123
1,Bodega comercial,250.683849
2,Casa,193.814888
3,Casa en condominio,186.547166
4,Casa uso de suelo,241.857895


In [189]:
test = test.merge(metros_mean, on = 'tipodepropiedad')

In [190]:
test_set = test_set.merge(test[['id', 'metroscubiertos_mean_tipodeprop']], on = "id", how = "left")

In [191]:
len(test_set)

60000

# Varianza de metroscubiertos por tipo de propiedad

In [192]:
metros_var = data.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "var"}).reset_index()

In [193]:
metros_var.rename(columns = {"metroscubiertos" : "metroscubiertos_var_tipodeprop"}, inplace = True)

In [194]:
data = data.merge(metros_var, on = 'tipodepropiedad')

In [195]:
train_set = train_set.merge(data[['id', 'metroscubiertos_var_tipodeprop']], on = "id")

### Lo mismo pero para test

In [196]:
metros_var = test.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "var"}).reset_index()

In [197]:
metros_var.rename(columns = {"metroscubiertos" : "metroscubiertos_var_tipodeprop"}, inplace = True)

In [198]:
test = test.merge(metros_var, on = 'tipodepropiedad')

In [199]:
test_set = test_set.merge(test[['id', 'metroscubiertos_var_tipodeprop']], on = "id", how = "left")

In [200]:
len(test_set)

60000

# Precio promedio según tipo de propiedad

In [201]:
precio_mean = data.groupby(by = 'tipodepropiedad').agg({"precio" : "mean"}).reset_index()

In [202]:
precio_mean.head()

,tipodepropiedad,precio
0,Apartamento,2.763769e+06
1,Bodega comercial,2.693758e+06
2,Casa,2.398158e+06
3,Casa en condominio,2.898927e+06
4,Casa uso de suelo,3.732469e+06


In [203]:
precio_mean.rename(columns = {"precio" : "precio_mean_tipodeprop"}, inplace = True)
precio_mean.head()

,tipodepropiedad,precio_mean_tipodeprop
0,Apartamento,2.763769e+06
1,Bodega comercial,2.693758e+06
2,Casa,2.398158e+06
3,Casa en condominio,2.898927e+06
4,Casa uso de suelo,3.732469e+06


In [204]:
data = data.merge(precio_mean, on = 'tipodepropiedad')

In [205]:
train_set = train_set.merge(data[['id', 'precio_mean_tipodeprop']], on = "id")

### Lo mismo pero para test

In [206]:
test = test.merge(precio_mean, on = 'tipodepropiedad')

In [207]:
test_set = test_set.merge(test[['id', 'precio_mean_tipodeprop']], on = "id", how = "left")

In [208]:
len(test_set)

60000

# Varianza del precio segun tipo de propiedad

In [209]:
precio_var = data.groupby(by = 'tipodepropiedad').agg({"precio" : "var"}).reset_index()

In [210]:
precio_var.head()

,tipodepropiedad,precio
0,Apartamento,5.488593e+12
1,Bodega comercial,4.009818e+12
2,Casa,3.997227e+12
3,Casa en condominio,5.359409e+12
4,Casa uso de suelo,6.400712e+12


In [211]:
precio_var.rename(columns = {"precio" : "precio_var_tipodeprop"}, inplace = True)
precio_var.head()

,tipodepropiedad,precio_var_tipodeprop
0,Apartamento,5.488593e+12
1,Bodega comercial,4.009818e+12
2,Casa,3.997227e+12
3,Casa en condominio,5.359409e+12
4,Casa uso de suelo,6.400712e+12


In [212]:
data = data.merge(precio_var, on = 'tipodepropiedad')

In [213]:
train_set = train_set.merge(data[['id', 'precio_var_tipodeprop']], on = "id")

### Lo mismo pero para test

In [214]:
test = test.merge(precio_var, on = 'tipodepropiedad')

In [215]:
test_set = test_set.merge(test[['id', 'precio_var_tipodeprop']], on = "id", how = "left")

In [216]:
len(test_set)

60000

# Precio promedio  para intervalos de metros

Da muy mal este feature

In [217]:
# bins = 10
# precio_mean_mts = data.groupby(by = pd.cut(data['metroscubiertos'], bins)).agg({"precio" : "mean"}).reset_index()
# data['metroscubiertos_bin'] = pd.cut(data["metroscubiertos"], bins)
# precio_mean_mts.head()

In [218]:
# precio_mean_mts.rename(columns = {"metroscubiertos":"metroscubiertos_bin", "precio":"precio_mean_mts"}, inplace = True)
# precio_mean_mts.head()

In [219]:
# data = data.merge(precio_mean_mts, on = 'metroscubiertos_bin')

In [220]:
# train_set = train_set.merge(data[['id', 'precio_mean_mts']], on = "id")

In [221]:
# precio_mean_mts['metroscubiertos_bin']

In [222]:
# bins = pd.IntervalIndex.from_tuples([(14.576, 57.4),
#       (57.4, 99.8),
#      (99.8, 142.2),
#     (142.2, 184.6),
#     (184.6, 227.0),
#     (227.0, 269.4),
#     (269.4, 311.8),
#     (311.8, 354.2),
#     (354.2, 396.6),
#     (396.6, 439.0)], closed = 'right' )
# test_set['metroscubiertos_bin'] = pd.cut(test_set['metroscubiertos'], bins)

In [223]:
# test_set['metroscubiertos_bin'].head()

In [224]:
# precio_mean_mts.head()

In [225]:
# test_set = test_set.merge(precio_mean_mts, on = "metroscubiertos_bin", how = 'left')

In [226]:
# test_set.head()

# Varianza de precio para intervalos de metros

Da muy mal este feature

In [227]:
# bins = 10
# precio_var_mts = data.groupby(by = pd.cut(data['metroscubiertos'], bins)).agg({"precio" : "var"}).reset_index()
# data['metroscubiertos_bin'] = pd.cut(data["metroscubiertos"], bins)
# precio_var_mts.head()

In [228]:
# precio_var_mts.rename(columns = {"metroscubiertos":"metroscubiertos_bin", "precio":"precio_var_mts"}, inplace = True)
# precio_var_mts.head()

In [229]:
# data = data.merge(precio_var_mts, on = 'metroscubiertos_bin')

In [230]:
# train_set = train_set.merge(data[['id', 'precio_var_mts']], on = "id")

In [231]:
# precio_var_mts['metroscubiertos_bin']

In [232]:
# bins = pd.IntervalIndex.from_tuples([(14.576, 57.4),
#       (57.4, 99.8),
#      (99.8, 142.2),
#     (142.2, 184.6),
#     (184.6, 227.0),
#     (227.0, 269.4),
#     (269.4, 311.8),
#     (311.8, 354.2),
#     (354.2, 396.6),
#     (396.6, 439.0)], closed = 'right' )
# test_set['metroscubiertos_bin'] = pd.cut(test_set['metroscubiertos'], bins)

In [233]:
# test_set['metroscubiertos_bin'].head()

In [234]:
# precio_var_mts.head()

In [235]:
# test_set = test_set.merge(precio_var_mts, on = "metroscubiertos_bin", how = 'left')

In [236]:
# test_set.head()

In [237]:
# del test_set['metroscubiertos_bin']

# Precio promedio por latitud y longitud

In [250]:
# bins = 10
# data['lat_bin'] = pd.cut(data["lat"], bins)
# data['lng_bin'] = pd.cut(data['lng'], bins)
# precio_mean_lat_lng = data.groupby(by = ['lat_bin', 'lng_bin']).agg({"precio" : "mean"}).reset_index()
# precio_mean_lat_lng.head()

In [251]:
# precio_mean_lat_lng.rename(columns = {"precio":"precio_mean_lat_lng_bin"}, inplace = True)
# precio_mean_lat_lng.head()S

In [252]:
# data = data.merge(precio_mean_lat_lng, on = ['lat_bin', 'lng_bin'])
# data['precio_mean_lat_lng_bin']

In [253]:
# train_set = train_set.merge(data[['id', 'precio_mean_lat_lng_bin']], on = "id")

In [254]:
# precio_mean_lat_lng['lat_bin'].unique()

In [255]:
# precio_mean_lat_lng['lng_bin'].unique()

In [256]:
# bins = pd.IntervalIndex.from_tuples([(-80.79, -64.261), (-64.261, -47.896), 
# (-47.896, -31.531), (-31.531, -15.165), 
# (-15.165, 1.2), (1.2, 17.565), 
# (17.565, 33.93), (33.93, 50.296), 
# (50.296, 66.661), (66.661, 83.026)], closed = 'right' )
# test_set['lat_bin'] = pd.cut(test_set['lat'], bins)

In [257]:
# bins = pd.IntervalIndex.from_tuples([(-126.106, -101.17), (-101.17, -76.48), 
# (-76.48, -51.791), (-51.791, -27.101), 
# (-27.101, -2.412), (-2.412, 22.278), 
# (22.278, 46.967), (46.967, 71.657), 
# (71.657, 96.346), (96.346, 121.036)], closed = 'right' )
# test_set['lng_bin'] = pd.cut(test_set['lng'], bins)

In [258]:
# test_set['lng_bin']

In [259]:
# test_set['lat_bin']

In [260]:
# precio_mean_lat_lng['lat_bin']

In [261]:
# test_set = test_set.merge(precio_mean_lat_lng, on = ["lat_bin"], how = 'left')

Por que lat_bin no anda pero lng_bin si? Si son exactamente el mismo tipo

In [262]:
# test_set.head()

In [263]:
# del test['lat_bin']
# del test['lng_bin']

# Se exporta para entrenar

## NO hace falta eliminar NULLs

In [ ]:
len(train_set)

In [ ]:
train_set.to_csv("train_set_xgb.csv", index = False)

In [ ]:
test_set.head()

In [ ]:
len(test_set)

In [ ]:
test_set.to_csv("test_set_xgb.csv", index = False)